In [ ]:
import pandas as pd
import numpy as np

In [ ]:
filename = '../data/data_merging/data/01_personal_data.csv';
personal_data = pd.DataFrame.from_csv(filename)

In [ ]:
personal_data2 = personal_data[[0,2]].copy()
personal_data2['klout_score'] = pd.Series(np.zeros(len(personal_data2)), index=personal_data2.index)
personal_data2['user_screen_name'] = pd.Series(['' for i in range(len(personal_data2))], index=personal_data2.index)

In [ ]:
customer_segment = []
with open('../data/data_merging/data/03_bank_fin_data.csv') as f:
    ff = 0
    for l in f:
        if ff == 0:
            ff = 1
            continue
        l2 = l.split(',')
        customer_segment.append([int(l2[0]),int(l2[9])])

In [ ]:
personal_data2['got_reply'] = pd.Series(np.zeros(len(personal_data2)), 
                                       index=personal_data2.index)

personal_data2['segment'] = pd.Series([v[1] for v in customer_segment], 
                                       index=[v[0] for v in customer_segment])

personal_data2['followers_count'] = pd.Series(np.zeros(len(personal_data2)), 
                                       index=personal_data2.index)

In [ ]:
customer_data = {'user_screen_name':'capgbanktest',
                 'segment':5,
                 'got_reply':0,
                 'state':'NY',
                 'klout_score':39,
                 'followers_count':1049,
                 'name':'John Snow'
                }
for k,v in customer_data.iteritems():
    personal_data2.loc[1,k] = v


In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql+psycopg2:///twitter?user=postgres')
#personal_data2[:1000].to_sql('customers', engine)

In [ ]:
stream_data = [{
    'id_str':'101',
    'categories':'|charges|',
    'in_reply_to_user_id':'',
    'sentiment':-2,
    'text':'Had double charge on my account $175.00 x2 for Bill Pay. My is now negative funds. App transaction error went and did it twice. #CapBankHelp',
    'created_at':'03-01-2016',
    'user_location':'NYC',
    'in_reply_to_status_id':'',
    'in_reply_to_screen_name':'',
    'retweet_count':0,
    'favorite_count':0,
    'user_name':'George Martin',
    'user_screen_name':'gmartin',
    'user_id_str':'1',
    'is_agent_reply':0
},{
    'id_str':'102',
    'categories':'',
    'in_reply_to_user_id':'',
    'sentiment':-2,
    'text':'Sorry to hear that, we fixed that and will make sure that it will not happen again',
    'created_at':'03-01-2016',
    'user_location':'NYC',
    'in_reply_to_status_id':'',
    'in_reply_to_screen_name':'',
    'retweet_count':0,
    'favorite_count':0,
    'user_name':'George Martin',
    'user_screen_name':'gmartin',
    'user_id_str':'1',
    'is_agent_reply':1
},{
    'id_str':'103',
    'categories':'',
    'in_reply_to_user_id':'',
    'sentiment':2,
    'text':'Thanks! Appreciate fast response!',
    'created_at':'03-01-2016',
    'user_location':'NYC',
    'in_reply_to_status_id':'',
    'in_reply_to_screen_name':'',
    'retweet_count':0,
    'favorite_count':0,
    'user_name':'George Martin',
    'user_screen_name':'gmartin',
    'user_id_str':'1',
    'is_agent_reply':0
},{
    'id_str':'104',
    'categories':'|support|',
    'in_reply_to_user_id':'',
    'sentiment':-2,
    'text':'Unbelievable. I was told I had to wait 3-5 business days to even speak to a technician.',
    'created_at':'04-01-2016',
    'user_location':'NYC',
    'in_reply_to_status_id':'',
    'in_reply_to_screen_name':'',
    'retweet_count':0,
    'favorite_count':0,
    'user_name':'Katie Hopkins',
    'user_screen_name':'khopkins',
    'user_id_str':'2',
    'is_agent_reply':0
},{
    'id_str':'105',
    'categories':'',
    'in_reply_to_user_id':'',
    'sentiment':-2,
    'text':'Hi, Katie. We apologize for the inconvenience. Let us call you.',
    'created_at':'04-01-2016',
    'user_location':'NYC',
    'in_reply_to_status_id':'',
    'in_reply_to_screen_name':'',
    'retweet_count':0,
    'favorite_count':0,
    'user_name':'Katie Hopkins',
    'user_screen_name':'khopkins',
    'user_id_str':'2',
    'is_agent_reply':1
},{
    'id_str':'106',
    'categories':'',
    'in_reply_to_user_id':'',
    'sentiment':2.5,
    'text':'Got a call from your team and my problem was solved in 5 minutes. Nice job!',
    'created_at':'04-01-2016',
    'user_location':'NYC',
    'in_reply_to_status_id':'',
    'in_reply_to_screen_name':'',
    'retweet_count':0,
    'favorite_count':0,
    'user_name':'Katie Hopkins',
    'user_screen_name':'khopkins',
    'user_id_str':'2',
    'is_agent_reply':0
}]

In [ ]:
customer_data = [{
        'index':2,
        'name':'George Martin',
        'user_screen_name':'gmartin',
        'got_reply':1,
        'segment':3
    },{
        'index':3,
        'name':'Katie Hopkins',
        'user_screen_name':'khopkins',
        'got_reply':1,
        'segment':4
    }]

In [ ]:
def insert_dict(d, table_name, engine):
    values = ['\''+v+'\'' if isinstance(v, str) else v for v in d.values()]
    command=("insert into " + table_name + " (" + (','.join(['%s']*len(d))) + ") "+
                "values (" + (','.join(['%s']*len(d))) + ");")%tuple(d.keys()+values)
    return engine.execute(command)

def update_dict(d, table_name, engine):
    values = ['\''+v+'\'' if isinstance(v, str) else v for v in d.values()]
    list_of_pairs = [item for pair in  zip(d.keys(),values) for item in pair]
    list_of_pairs = tuple(list_of_pairs)
    
    command=("update " + table_name + " set " + (','.join(['%s=%s']*len(d))) + " where index="+str(d['index']))%list_of_pairs
    return engine.execute(command)

In [ ]:
engine.execute('delete from stream;')
for tweet in stream_data:
    insert_dict(tweet, 'stream', engine)

for c in customer_data:
    update_dict(c, 'customers', engine)